In [32]:
import sys
import os
import pandas as pd
sys.path.append("/Users/natwat/Desktop/CPSC_Projects/InvBankAI/")
PROJECT_ROOT = "/Users/natwat/Desktop/CPSC_Projects/InvBankAI/"

In [33]:

from combineSingleDB import load_and_aggregate_sentiment, full_incremental_merge, full_merge
# Paths
sent_path = "/Users/natwat/Desktop/CPSC_Projects/InvBankAI/data/news/TSLA_sentiment_combined.csv"
price_path = "/Users/natwat/Desktop/CPSC_Projects/InvBankAI/data/features/TSLA_features.csv"  # Must be minute-level
output_path = "/Users/natwat/Desktop/CPSC_Projects/InvBankAI/data/features/TSLA_features_full.csv"
archive_path = "/Users/natwat/Desktop/CPSC_Projects/INVBANKAI/data/final/TSLA_Archive.csv"


# Run full pipeline
sentiment_df = load_and_aggregate_sentiment(sent_path, resample_interval="1min")
sentiment_df.tail()


,Datetime,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,Prob_Pos_roberta,Prob_Neu_roberta,Prob_Neg_roberta,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,Sentiment_Avg,Sentiment_Disagreement
11009,2025-07-14 18:14:00+00:00,8.676738e-07,9.999981e-01,0.000001,0.900163,0.097327,0.002511,0.999637,0.0,0.000363,0.633267,0.550675
11010,2025-07-14 18:15:00+00:00,8.676738e-07,9.999981e-01,0.000001,0.900163,0.097327,0.002511,0.999637,0.0,0.000363,0.633267,0.550675
11011,2025-07-14 18:16:00+00:00,8.676738e-07,9.999981e-01,0.000001,0.900163,0.097327,0.002511,0.999637,0.0,0.000363,0.633267,0.550675
11012,2025-07-14 18:17:00+00:00,8.676738e-07,9.999981e-01,0.000001,0.900163,0.097327,0.002511,0.999637,0.0,0.000363,0.633267,0.550675
11013,2025-07-14 18:18:00+00:00,1.695669e-07,5.004997e-08,1.000000,0.080077,0.910761,0.009162,0.993390,0.0,0.006610,0.357822,0.551872


In [34]:

from combineSingleDB import full_merge
# Get last timestamp in existing merged file
if os.path.exists(price_path):
    df = pd.read_csv(price_path, parse_dates=["Datetime"])
    last_ts = df["Datetime"].max()
    
else:
    last_ts = None

# Run the updated merge
full_incremental_merge(sentiment_df, price_path, output_path)

# View output
df = pd.read_csv(output_path)
df.tail()

⏸️ No new price data to merge.


,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,Prob_Pos_roberta,Prob_Neu_roberta,Prob_Neg_roberta,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,Sentiment_Avg,Sentiment_Disagreement,Return,Label
2317,2025-07-14 19:55:00+00:00,316.790009,317.000000,316.550110,316.750000,428902.0,316.435484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000316,0
2318,2025-07-14 19:56:00+00:00,316.690002,316.929901,316.589996,316.779999,260013.0,316.459724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000469,1
2319,2025-07-14 19:57:00+00:00,316.838593,316.890015,316.690002,316.700012,284435.0,316.495807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000762,1
2320,2025-07-14 19:58:00+00:00,317.079987,317.109985,316.820007,316.829987,486575.0,316.551443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000536,0
2321,2025-07-14 19:59:00+00:00,316.910004,317.269989,316.809998,317.079987,666850.0,316.585592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [35]:
import ta

df = pd.read_csv("/Users/natwat/Desktop/CPSC_Projects/InvBankAI/data/features/TSLA_features_full.csv", parse_dates=["Datetime"])

# Add rolling indicators
df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
df["RSI_14"] = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
df["MACD"] = ta.trend.macd_diff(df["Close"])
vwap = ta.volume.VolumeWeightedAveragePrice(
    high=df["High"], low=df["Low"], close=df["Close"], volume=df["Volume"]
)
df["VWAP"] = vwap.vwap  
df.tail()



,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,...,Prob_Neu_distilbert,Prob_Neg_distilbert,Sentiment_Avg,Sentiment_Disagreement,Return,Label,SMA_50,RSI_14,MACD,VWAP
2317,2025-07-14 19:55:00+00:00,316.790009,317.000000,316.550110,316.750000,428902.0,316.435484,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.000316,0,316.253805,58.760430,0.041726,316.500199
2318,2025-07-14 19:56:00+00:00,316.690002,316.929901,316.589996,316.779999,260013.0,316.459724,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000469,1,316.262207,56.351438,0.035907,316.531738
2319,2025-07-14 19:57:00+00:00,316.838593,316.890015,316.690002,316.700012,284435.0,316.495807,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000762,1,316.272579,59.038480,0.038916,316.562523
2320,2025-07-14 19:58:00+00:00,317.079987,317.109985,316.820007,316.829987,486575.0,316.551443,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.000536,0,316.290378,63.021191,0.053134,316.628758
2321,2025-07-14 19:59:00+00:00,316.910004,317.269989,316.809998,317.079987,666850.0,316.585592,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,316.303578,58.693501,0.047258,316.699203


In [36]:
df["Return_1min"] = df["Close"].pct_change()
df["Volume_Change"] = df["Volume"].pct_change()
df["Price_Diff"] = df["Close"] - df["Open"]
df.to_csv("/Users/natwat/Desktop/CPSC_Projects/INVBANKAI/data/final/TSLA_ready.csv", index=False)
df.tail()

,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,...,Sentiment_Disagreement,Return,Label,SMA_50,RSI_14,MACD,VWAP,Return_1min,Volume_Change,Price_Diff
2317,2025-07-14 19:55:00+00:00,316.790009,317.000000,316.550110,316.750000,428902.0,316.435484,NaN,NaN,NaN,...,NaN,-0.000316,0,316.253805,58.760430,0.041726,316.500199,0.000095,0.566874,0.040009
2318,2025-07-14 19:56:00+00:00,316.690002,316.929901,316.589996,316.779999,260013.0,316.459724,NaN,NaN,NaN,...,NaN,0.000469,1,316.262207,56.351438,0.035907,316.531738,-0.000316,-0.393771,-0.089996
2319,2025-07-14 19:57:00+00:00,316.838593,316.890015,316.690002,316.700012,284435.0,316.495807,NaN,NaN,NaN,...,NaN,0.000762,1,316.272579,59.038480,0.038916,316.562523,0.000469,0.093926,0.138580
2320,2025-07-14 19:58:00+00:00,317.079987,317.109985,316.820007,316.829987,486575.0,316.551443,NaN,NaN,NaN,...,NaN,-0.000536,0,316.290378,63.021191,0.053134,316.628758,0.000762,0.710672,0.250000
2321,2025-07-14 19:59:00+00:00,316.910004,317.269989,316.809998,317.079987,666850.0,316.585592,NaN,NaN,NaN,...,NaN,NaN,0,316.303578,58.693501,0.047258,316.699203,-0.000536,0.370498,-0.169983


In [37]:
# Drop rows with NaN from rolling indicators
df.dropna(inplace=True)

# Define your input features
feature_cols = [
    "Close", "High", "Low", "Open", "Volume", "EMA20", "SMA_50", "RSI_14", "MACD", "VWAP",
    "Price_Diff", "Return_1min", "Volume_Change",
    "Prob_Pos_distilbert", "Prob_Neu_distilbert", "Prob_Neg_distilbert",
    "Prob_Pos_roberta", "Prob_Neu_roberta", "Prob_Neg_roberta",
    "Prob_Pos_finbert", "Prob_Neu_finbert", "Prob_Neg_finbert",
    "Sentiment_Avg", "Sentiment_Disagreement"
]

# Your label (used later for evaluation, not part of input state)
label_col = "Label"

In [38]:
df.to_csv("/Users/natwat/Desktop/CPSC_Projects/INVBANKAI/data/final/TSLA_ready.csv", index=False)

# Try to load archive, else start fresh
if os.path.exists(archive_path):
    archive_df = pd.read_csv(archive_path, parse_dates=["Datetime"])
    # Concatenate, remove duplicates, and sort
    combined = pd.concat([archive_df,df])
    combined = combined.drop_duplicates(subset=["Datetime"],keep="last").sort_values("Datetime").reset_index(drop=True)
else:
    combined = df.copy()
# Save back to archive
combined.to_csv(archive_path, index=False)


,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,...,Sentiment_Disagreement,Return,Label,SMA_50,RSI_14,MACD,VWAP,Return_1min,Volume_Change,Price_Diff
1926,2025-07-11 19:54:00+00:00,313.559998,313.709991,313.100006,313.690002,306541.0,313.419318,0.938557,0.047027,0.014416,...,0.469072,0.000515,1,312.793815,53.503516,-0.024470,313.474271,-0.000335,0.239180,-0.130005
1927,2025-07-11 19:55:00+00:00,313.721405,313.750000,313.239990,313.559998,281822.0,313.448088,0.938557,0.047027,0.014416,...,0.469072,-0.000467,0,312.854843,55.498584,-0.019233,313.497728,0.000515,-0.080638,0.161407
1928,2025-07-11 19:56:00+00:00,313.575012,313.969910,313.450012,313.730011,577781.0,313.460176,0.938557,0.047027,0.014416,...,0.469072,-0.000303,0,312.915944,53.266193,-0.026317,313.529570,-0.000467,1.050163,-0.154999
1929,2025-07-11 19:57:00+00:00,313.480011,313.629486,313.350006,313.579987,244071.0,313.462065,0.938557,0.047027,0.014416,...,0.469072,-0.000160,0,312.968944,51.809749,-0.037413,313.541540,-0.000303,-0.577572,-0.099976
1930,2025-07-11 19:58:00+00:00,313.429993,313.500000,313.339996,313.484985,296555.0,313.459011,0.938557,0.047027,0.014416,...,0.469072,0.000223,1,313.015744,51.018785,-0.047485,313.547352,-0.000160,0.215036,-0.054993


In [39]:
df.tail()

,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,...,Sentiment_Disagreement,Return,Label,SMA_50,RSI_14,MACD,VWAP,Return_1min,Volume_Change,Price_Diff
1926,2025-07-11 19:54:00+00:00,313.559998,313.709991,313.100006,313.690002,306541.0,313.419318,0.938557,0.047027,0.014416,...,0.469072,0.000515,1,312.793815,53.503516,-0.024470,313.474271,-0.000335,0.239180,-0.130005
1927,2025-07-11 19:55:00+00:00,313.721405,313.750000,313.239990,313.559998,281822.0,313.448088,0.938557,0.047027,0.014416,...,0.469072,-0.000467,0,312.854843,55.498584,-0.019233,313.497728,0.000515,-0.080638,0.161407
1928,2025-07-11 19:56:00+00:00,313.575012,313.969910,313.450012,313.730011,577781.0,313.460176,0.938557,0.047027,0.014416,...,0.469072,-0.000303,0,312.915944,53.266193,-0.026317,313.529570,-0.000467,1.050163,-0.154999
1929,2025-07-11 19:57:00+00:00,313.480011,313.629486,313.350006,313.579987,244071.0,313.462065,0.938557,0.047027,0.014416,...,0.469072,-0.000160,0,312.968944,51.809749,-0.037413,313.541540,-0.000303,-0.577572,-0.099976
1930,2025-07-11 19:58:00+00:00,313.429993,313.500000,313.339996,313.484985,296555.0,313.459011,0.938557,0.047027,0.014416,...,0.469072,0.000223,1,313.015744,51.018785,-0.047485,313.547352,-0.000160,0.215036,-0.054993
